In [45]:
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb

In [46]:
data_df = pd.read_csv("data.csv")

In [47]:
data_df = data_df.drop(columns=['Age_x', 'Unnamed: 0.1', 'Unnamed: 0', 'CIF_CLSCUS', 'COB_DATE', 'DATE_TIME', 'BRN_OPN_CIF', 'MA_PHONG_GIAO_DICH_VCB', 'CIF_MASK'])

# Separate numerical and categorical columns in data.csv
numerical_vars = data_df.select_dtypes(include=['number']).columns
# to_scale_vars = numeric_vars.drop(columns = [''])
categorical_vars = data_df.select_dtypes(exclude=['number']).columns
data_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# data_df.replace([np.inf, -np.inf], np.nan, inplace=True)


# def preprocess_data(df):
#     # Separate categorical and numerical columns
#     categorical_cols = df.select_dtypes(include=['object']).columns
#     numerical_cols = df.select_dtypes(exclude=['object']).columns
    
#     # Preprocess with OneHotEncoder and StandardScaler
#     preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', Pipeline([
#             ('imputer', SimpleImputer(strategy='constant', fill_value=-9999999999999999999999)),
#             ('scaler', StandardScaler())
#         ]), numerical_cols),
        
#         ('cat', Pipeline([
#             ('imputer', SimpleImputer(strategy='constant', fill_value='None')),
#             # ('imputer', SimpleImputer(strategy='most_frequent')),
#             ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
#         ]), categorical_cols)
#         ])
    
#     processed_data = preprocessor.fit_transform(df)
#     return processed_data, preprocessor



# Split features and target
X = data_df.drop(columns=['IS_BANCAS'])
y = data_df['IS_BANCAS']


# Handle categorical variables
# For simplicity, using one-hot encoding, though other methods (target encoding) could be considered
X = pd.get_dummies(X, drop_first=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the XGBoost classifier
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', missing=np.nan)

# Fit the model
xgb_clf.fit(X_train, y_train)
# Make predictions
y_pred = xgb_clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
accuracy = round(accuracy, 4)
print("Model Accuracy:", str(accuracy*100) + "%")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Predict on a new observation
# Replace with a new observation, ensuring it matches the training feature format
new_observation = X_test.iloc[0]  # Example of using an observation from test set
# new_pred = xgb_clf.predict(np.array([new_observation]))
# print("Prediction for new observation (IS_BANCAS):", new_pred[0])

/usr/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [11:42:20] WARNING: /usr/src/debug/python-xgboost/xgboost-2.1.2/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Model Accuracy: 96.15%

Classification Report:
               precision    recall  f1-score   support

         0.0       0.97      0.99      0.98    207338
         1.0       0.71      0.32      0.44     10353

    accuracy                           0.96    217691
   macro avg       0.84      0.66      0.71    217691
weighted avg       0.95      0.96      0.95    217691

